In [6]:
import requests
import csv

def fetch_data(url, params):
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def extract_data(data):
    extracted_info = []
    body = data.get('body', [])
    
    for item in body:
        if item.get('RMV_YMD') is None:
            extracted_info.append({
                'DST_RSK_DSTRCT_NM': item.get('DST_RSK_DSTRCT_NM', ''), 
                'DST_RSK_DSTRCT_GRD_CD': item.get('DST_RSK_DSTRCT_GRD_CD', ''),
                'DST_RSK_DSTRCT_TYPE_CD': item.get('DST_RSK_DSTRCT_TYPE_CD', ''),
                'DST_RSK_DSTRCTCD': item.get('DST_RSK_DSTRCTCD', ''), 
                'DST_RSK_DSTRCT_RGN_CD': item.get('DST_RSK_DSTRCT_RGN_CD', ''),
                'FCLT_NM': item.get('FCLT_NM', ''),
                'DSGN_YMD': item.get('DSGN_YMD', ''),
                'DSGN_RSN': item.get('DSGN_RSN', ''),
                'RSK_FACTR_CN': item.get('RSK_FACTR_CN', ''),
                'DSGN_AREA': item.get('DSGN_AREA', ''),
                'STDG_CD': item.get('STDG_CD', ''),
                'DADDR': item.get('DADDR', ''),
                'RONA_DADDR': item.get('RONA_DADDR', '')
            })
    
    return extracted_info

def save_to_csv(data, file_path):
    with open(file_path, 'w', newline='', encoding='cp949') as csvfile:
        fieldnames = [
            'DST_RSK_DSTRCT_NM', 'DST_RSK_DSTRCT_GRD_CD', 'DST_RSK_DSTRCT_TYPE_CD',
            'DST_RSK_DSTRCTCD', 'DST_RSK_DSTRCT_RGN_CD', 'FCLT_NM', 'DSGN_YMD',
            'DSGN_RSN', 'RSK_FACTR_CN', 'DSGN_AREA', 'STDG_CD', 'DADDR', 'RONA_DADDR'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Example usage
base_url = 'https://www.safetydata.go.kr/V2/api/DSSP-IF-00058'
params = {'serviceKey': 'Z37IF8F5ETG9UVQ9', 'numOfRows': 10}

all_data = []
total_pages = 53  # Assuming you know the total number of pages

for page in range(1, total_pages + 1):
    params['pageNo'] = page
    data = fetch_data(base_url, params)
    extracted_info = extract_data(data)
    all_data.extend(extracted_info)

csv_file_path = './data/crisis(cp949).csv'  # Change to your desired output CSV path
save_to_csv(all_data, csv_file_path)
